# Install Python Dependencies
Run the following code cell to install the required python packages.
Note: your python environment should be python 3.8 or newer.

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


# Next, download the annotated data from the ICDAR 2024 MapText competition.

In [ ]:
!wget https://zenodo.org/records/11516933/files/rumsey_train.json
!wget https://zenodo.org/records/11516933/files/rumsey_val.json

# Download the Map Images
Optionally, you can download the source map images for these annotations. These images take up roughly 2 GB of disk storage, and you can download them by running the following cells:

In [ ]:
!wget https://zenodo.org/records/11516933/files/train.zip
!unzip train.zip
!wget https://zenodo.org/records/11516933/files/val.zip
!unzip val.zip

In [6]:
# run this cell to delete the .zip files for the previous download
!rm -rf train.zip
!rm -rf val.zip

# Construct and Evaluate Linkage Graphs
First, fill in the variables in the cell below to select which linkage graph method you want to use. Then, run the cell to construct linkage graphs using that method, and evaluate the results.

In [2]:
import sys
import os
import json
import random
sys.path.append(os.getcwd() + "/scripts")
from map_graph import FeatureNode, prims_mst, distance_threshold_graph, MahalanobisMetric
from compare_linkages import map_list_compare_linkages, get_stats_from_results_file, LinkageMethod
from cross_validation_mahalanobis import train_mahalanobis_metric_for_fold
map_ids = []
# change MAP_SAMPLE_SIZE to change the number of maps you randomly select.
MAP_SAMPLE_SIZE = 10
# to use the validation maps instead, change "annotations_filepath" from
# "rumsey_train.json" to "rumsey_val.json"
annotations_filepath = "rumsey_train.json"
with open(annotations_filepath, "r") as f:
    for map_annotation in json.load(f):
        map_ids.append(map_annotation["image"])
    map_sample = random.sample(map_ids, MAP_SAMPLE_SIZE)
    # you can change the linkage method here by changing the "linkage_method" variable
    # use the following variable for the linkage method presented in our paper
    linkage_method = LinkageMethod(prims_mst, FeatureNode.EdgeCostFunction([1, 1, 1]))
    # Uncomment the following lines to create MSTs using the Mahalanobis distance
    # note: training the Mahalanobis metric will take several minutes.
    """ M = train_mahalanobis_metric_for_fold([map_id for map_id in map_ids if map_id not in map_sample], "random_sample", annotations_filepath)
    linkage_method = LinkageMethod(prims_mst, MahalanobisMetric(M)) """
    # set the linkage_method as follows to create character distance threshold linkage graphs
    #linkage_method = LinkageMethod(distance_threshold_graph, FeatureNode.Distance)

    # name the output file for your method by setting the "name" variable.
    # this output file will be stored in a file called "results"
    name = "train_edge_cost_heuristic_metric"
    output_file_name = "results/" + name + ".json"
    # Create the "results" folder if it doesn't exist
    os.makedirs("results", exist_ok=True)
    map_list_compare_linkages(map_sample, name, annotations_filepath, linkage_method, output_file_name)
    get_stats_from_results_file(output_file_name, annotations_filepath)

initializing dict
loading annotations


KeyboardInterrupt: 

# Visualizing Linkage Graphs
To visualize the linkage graphs drawn on various maps, you can run the following code cell.
Note: you will need to have downloaded the map images in order to create these visualizations.

In [1]:
import sys
import os
import json
sys.path.append(os.getcwd() + "/scripts")
import draw_features_and_linkages as dfl
import map_graph
import multiword_name_extraction

# change this variable to choose which map file you want to visualize
map_filename = "rumsey/train/8179000_h6_w11.png"
mg = map_graph.MapGraph(map_filename)
map_graph.prims_mst(mg.nodes, map_graph.FeatureNode.EdgeCostFunction([1, 1, 1]))
map_annotations = multiword_name_extraction.extract_map_data_from_all_annotations(map_filename)
dfl.draw_features_and_linkages(map_filename, mg, "equation_1.png", map_dir=os.getcwd(), show_image=True)
mg = map_graph.MapGraph(map_filename)
map_graph.distance_threshold_graph(mg.nodes)
map_annotations = multiword_name_extraction.extract_map_data_from_all_annotations(map_filename)
dfl.draw_features_and_linkages(map_filename, mg, "character_distance_threshold.png",map_dir=os.getcwd(), show_image=True)

initializing dict
loading annotations
47


: 